In [3]:
import pandas as pd
from gensim import corpora, models
from googlesearch import search
import re
import chardet

In [4]:
# Lets check what encoding we have for our NSFdata files
NSF_csv_files = ['NSFdata/NSF_CCF.csv', 'NSFdata/NSF_CICI.csv', 'NSFdata/NSF_CSSI.csv', 'NSFdata/NSF_DIBBS.csv', 'NSFdata/NSF_MRI.csv', 'NSFdata/NSF_OAC.csv', 'NSFdata/NSF_SI2.csv']
for file in NSF_csv_files:
    with open(file, 'rb') as f:
        result = chardet.detect(f.read())
        print(file, result)

NSFdata/NSF_CCF.csv {'encoding': 'ISO-8859-1', 'confidence': 0.7299916171744574, 'language': ''}
NSFdata/NSF_CICI.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_CSSI.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_DIBBS.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_MRI.csv {'encoding': 'ISO-8859-1', 'confidence': 0.7299962504897843, 'language': ''}
NSFdata/NSF_OAC.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}
NSFdata/NSF_SI2.csv {'encoding': 'ISO-8859-1', 'confidence': 0.73, 'language': ''}


In [5]:
# 1. Read the CSV file and load it into a DataFrame
data = pd.read_csv('NSFdata/NSF_DIBBS.csv', encoding='ISO-8859-1')

In [7]:
projects = data[["AwardNumber", "Title", "NSFOrganization", "PrincipalInvestigator", "PIEmailAddress", "Abstract"]]
print(projects.head())

     AwardNumber                                              Title  \
0        1724889  CIF21 DIBBs: PD: Enhancing and Personalizing E...   
1        1443019  CIF21 DIBBs: DIBBs for Intelligence and Securi...   
2        1443037  CIF21 DIBBs: Collaborative Research: Cyberinfr...   
3        1719477  Cybersecurity Big Data and Analytics Sharing P...   
4        1255781  CIF21 DIBBs: Conceptualization of the Social a...   
..           ...                                                ...   
125      9813442  Seasonal Differences in Air-Snow Chemical Rela...   
126      9813549  Seasonal Differences in Air-Snow Chemical Rela...   
127      9907623  Investigation of Photochemical Transformations...   
128      1049021  Collaborative Research:  Type 1: LOI: L0217030...   
129      9907434  Investigation of Photochemical Tranformations ...   

    NSFOrganization PrincipalInvestigator           PIEmailAddress  \
0               OAC        Neil Heffernan              nth@wpi.edu   
1      